In [152]:
from datetime import datetime
from os import getcwd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from UliPlot.XLSX import auto_adjust_xlsx_column_width

In [153]:
# T1
cooper = pd.read_excel('cooper.xls')
ernest = pd.read_excel('ernest.xls')
aalto = pd.read_excel('aalto.xls')
leo = pd.read_excel('leo.xls')
howard = pd.read_excel('howard.xls')
#T3
jason = pd.read_excel('jason.xls')
chris = pd.read_excel('chris.xls')
wendy = pd.read_excel('wendy.xls')
evelyn = pd.read_excel('evelyn.xls')

In [154]:
pmsystem_cft = (pd.concat([cooper,ernest,aalto,leo,howard,jason,chris,wendy,evelyn])
.iloc[:,[0,44]]
.set_index('Project ID')
)

In [155]:
# cft_list_file = '(Input) CFTListExport.xls'
# pmsystem_cft = pd.read_excel(cft_list_file, usecols=[0,44], index_col=[0])

In [156]:
pmsystem_project_list_file = '(Input) ProjectListExport.xls'
prweb_file = '(Input) New Production Status Summary.xlsx'

start_from = '2023-06'
consumer_dpqe_list = ['Lo, Aalto','Kuo, Chris','Wang, Cooper', 'Fu, Ernest','Hsu, Jason(PQE)','Ni, Wendy','Wang, Evelyn','Hsu, Jason(PQE);\nWang, Evelyn','Chen, Leo(PQE)','Jen, Howard'] 
# 'Hsu, Jason(PQE);\nWang, Evelyn' is for GPSMAP 9000 series

# PM system

Read

In [157]:
pmsystem_project = pd.read_excel(pmsystem_project_list_file, usecols=[0,1,3,4,5,6,9,10,12], parse_dates=[2,3,4,5], index_col=[0], engine='openpyxl')
# pmsystem_cft = pd.read_excel(cft_list_file, usecols=[0,44], index_col=[0])

c:\Users\wangcooper\Miniconda3\envs\workspace3.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [158]:
pmsystem_project.drop([14704], inplace=True)

In [159]:
pmsystem_project = pmsystem_project.query("~`Project Name`.str.contains('BMW')")

In [160]:
pmsystem_project = pmsystem_project[(pmsystem_project['PDP/Form107'] == 'PDP')]

In [161]:
pmsystem_project.head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status
Project ID,,,,,,,,
16544,2024 Value Dash Cams,2023-06-30,2023-10-23,2024-03-18,2024-01-12,Global,PDP,Development
15624,Catalyst Kart/Auto,2022-06-24,2023-02-24,2023-10-23,2023-08-03,Global,PDP,Development
16199,DC-2023 Premium Dash Cameras,2023-02-02,NaT,2023-08-28,2023-06-30,Global,PDP,Development
16718,DEZL 2024,2023-08-18,2024-01-16,2024-06-10,2024-04-09,Global,PDP,Development
16249,dezl headset (2nd Gen),2023-06-05,NaT,2024-05-06,2024-02-19,Global,PDP,Development


Merge project and CFT lists

In [162]:
pmsystem_project_cft = pd.merge(left=pmsystem_project, right=pmsystem_cft, left_index=True, right_index=True)
pmsystem_project_cft.sort_index().head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,
2854,Product - GDC 7500,2013-04-30,NaT,2019-07-01,2018-10-12,Global,PDP,Development,"Kuo, Chris"
5711,Product - GLA 85,2014-10-30,NaT,2018-12-07,2016-08-31,Global,PDP,Development,"Kuo, Chris"
7029,Alpha Dog Collar,2021-11-18,2022-08-18,2023-04-10,2023-01-19,Global,PDP,Development,"Kuo, Chris"
7563,Mercury,NaT,NaT,2022-01-17,2021-04-23,Global,PDP,Development,"Fu, Ernest"
7899,Product - GI 275,2018-04-04,NaT,2019-04-02,2019-03-22,Global,PDP,Development,"Kuo, Chris"


In [163]:
# (pmsystem_project_cft
#  .rename(columns={'Develop  Product QE (DPQE)':'DPQE'})
#  .query("DPQE.str.contains(';')")
# )

In [164]:
# (pmsystem_project_cft
#  .rename(columns={'Develop  Product QE (DPQE)':'DPQE'})
#  .query("DPQE == 'Fu, Ernest'")
# )

Find projects which WW/ASIA SKU have different DPQE

In [165]:
ww_asia_diff = pmsystem_project_cft.copy()

In [166]:
#Handle Asis words in 'Project Name' column
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].apply(lambda name : name.split(',')[0])
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].str.replace('ASIA','')
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].str.replace('Asia','')

In [167]:
# check any missing
ww_asia_diff[ww_asia_diff['Project Name'].str.contains('ASIA|asia|Asia')]

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,


In [168]:
ww_asia_diff = ww_asia_diff[['Project Name','Develop  Product QE (DPQE)','Global/Asia','MP Date']].loc[ww_asia_diff['MP Date']> start_from]

In [169]:
# Find duplicated projects
ww_asia_diff = ww_asia_diff[ww_asia_diff.duplicated(subset=['Project Name'],keep=False)]
ww_asia_diff

,Project Name,Develop Product QE (DPQE),Global/Asia,MP Date
Project ID,,,,
9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Global,2023-07-31
9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Global,2023-07-31
10901,eTrex Solar,"Kuo, Chris",Global,2023-10-11
10926,Approach R50,"Kuo, Chris",Global,2024-03-04
12799,Descent S1 Buoy,"Lo, Aalto",Global,2024-03-11
12994,Descent Mk3,"Wang, Cooper",Global,2023-10-23
14069,Descent S1 Buoy,"Lo, Aalto",Asia,2024-03-11
15509,Approach R50,"Kuo, Chris",Asia,2024-03-04
15519,Venu 3S,"Chen, Leo(PQE)",Global,2023-07-03


In [170]:
# Find projects have different DPQE
ww_asia_diff_DPQE = ww_asia_diff[~ww_asia_diff.duplicated(subset=['Project Name','Develop  Product QE (DPQE)'],keep=False)]

Find projects w/o DPQE (include ASIA SKUs)

In [171]:
pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isnull()].dropna(thresh=5, axis=0)

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,


In [172]:
projects_wo_DPQE = pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isnull()].dropna(thresh=5, axis=0).query("`MP Date` > @start_from")

Find projects with multi-DPQE (include ASIA SKUs)

In [173]:
multi_dpqe = pmsystem_project_cft[['Project Name','Global/Asia','Develop  Product QE (DPQE)','MP Date']]
multi_dpqe = multi_dpqe[multi_dpqe['MP Date']>start_from]
multi_dpqe.dropna(inplace=True)

In [174]:
multi_dpqe = multi_dpqe[multi_dpqe['Develop  Product QE (DPQE)'].str.contains('\n')]

Find projects w/o any date

In [175]:
# Find all date is NaT
# 1. Select dataframe w/ date (pmsystem_project_cft.iloc[:,1:5])
# 2. Rows w/ any date marked as True (notnull().any(axis=1))
# 3. Reverse (~)
pmsystem_project_wo_any_date = pmsystem_project_cft.loc[(~pmsystem_project_cft.iloc[:,1:5].notnull().any(axis=1))].reset_index().sort_values(['Develop  Product QE (DPQE)','Project Name']).set_index('Develop  Product QE (DPQE)')
pmsystem_project_wo_any_date

,Project ID,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status
Develop Product QE (DPQE),,,,,,,,,
"Fu, Ernest",10692,SaMD Sleep Apnea Risk Assessment,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Hsu, Jason(PQE)",16753,Fusion - 80 Series Marine Stereos,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Hsu, Jason(PQE)",8752,Marine inReach Antenna,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Hsu, Jason(PQE)",16501,Panoptix PS22-IF Transducer and Ice Fishing Bu...,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Jen, Howard",16935,EPIX (Gen 3) LTE,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",16384,"2"" Thin Watch",NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",13036,Alpha Max,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",13034,Companion Dog Tracking Collar,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",14604,L20 Laser Range Finder,NaT,NaT,NaT,NaT,Global,PDP,Concepts


Melt

In [176]:
df_consumer_dpqe = pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isin(consumer_dpqe_list)].sort_index()
df_consumer_dpqe = df_consumer_dpqe[df_consumer_dpqe['Global/Asia']=='Global']
df_consumer_dpqe.head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,
2854,Product - GDC 7500,2013-04-30,NaT,2019-07-01,2018-10-12,Global,PDP,Development,"Kuo, Chris"
5711,Product - GLA 85,2014-10-30,NaT,2018-12-07,2016-08-31,Global,PDP,Development,"Kuo, Chris"
7029,Alpha Dog Collar,2021-11-18,2022-08-18,2023-04-10,2023-01-19,Global,PDP,Development,"Kuo, Chris"
7563,Mercury,NaT,NaT,2022-01-17,2021-04-23,Global,PDP,Development,"Fu, Ernest"
7899,Product - GI 275,2018-04-04,NaT,2019-04-02,2019-03-22,Global,PDP,Development,"Kuo, Chris"


In [177]:
value_vars = ['Working Prototype Built and Tested','Final Factory Prototypes Built','PR Date','MP Date']
pmsystem_project_dpqe_melted = df_consumer_dpqe.reset_index().melt(id_vars= ['Project ID','Project Name','Develop  Product QE (DPQE)'],value_vars=value_vars)
pmsystem_project_dpqe_melted.head()

,Project ID,Project Name,Develop Product QE (DPQE),variable,value
0,2854,Product - GDC 7500,"Kuo, Chris",Working Prototype Built and Tested,2013-04-30
1,5711,Product - GLA 85,"Kuo, Chris",Working Prototype Built and Tested,2014-10-30
2,7029,Alpha Dog Collar,"Kuo, Chris",Working Prototype Built and Tested,2021-11-18
3,7563,Mercury,"Fu, Ernest",Working Prototype Built and Tested,NaT
4,7899,Product - GI 275,"Kuo, Chris",Working Prototype Built and Tested,2018-04-04


In [178]:
pmsystem_project_dpqe_melted.rename(columns={'Develop  Product QE (DPQE)':'DPQE','variable':'Phase','value':'Date'},inplace=True)

In [179]:
pmsystem_project_dpqe_melted.Date = pmsystem_project_dpqe_melted.Date.dt.strftime('%Y-%m')

In [180]:
pmsystem_project_dpqe_melted = pmsystem_project_dpqe_melted[(pmsystem_project_dpqe_melted.Date.isnull()) | (pmsystem_project_dpqe_melted.Date>=start_from)]

In [181]:
pmsystem_project_dpqe_melted.isnull().sum()

Project ID        0
Project Name      0
DPQE              0
Phase             0
Date            107
dtype: int64

Weight socre

In [182]:
pmsystem_project_dpqe_melted.loc[:,'weighted_score'] = pmsystem_project_dpqe_melted.loc[:,'Phase'].apply(lambda phase : 1 if phase == 'Working Prototype Built and Tested' else 2 if phase=='Final Factory Prototypes Built' else 3 if phase=='PR Date' else 4 if phase == 'MP Date' else 0)

In [183]:
pmsystem_project_dpqe_melted = pmsystem_project_dpqe_melted.rename(columns={"Project ID":'PM Project Id','Project Name':'PM Project Name'})

In [184]:
pmsystem_project_dpqe_melted.head()

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score
3,7563,Mercury,"Fu, Ernest",Working Prototype Built and Tested,NaN,1
6,8752,Marine inReach Antenna,"Hsu, Jason(PQE)",Working Prototype Built and Tested,NaN,1
12,10617,Force Kayak Propulsion System,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2023-10,1
13,10692,SaMD Sleep Apnea Risk Assessment,"Fu, Ernest",Working Prototype Built and Tested,NaN,1
16,11252,MARQ Carbon (Gen 2),"Jen, Howard",Working Prototype Built and Tested,NaN,1


# PRweb

Read

In [185]:
prweb = pd.read_excel(prweb_file, header=1, usecols=[0,3])
prweb = prweb[prweb['Project Name'] != 'GPSMAP 9000_Black Box'] # 2022/09/05, GPSMAP 9500 replace GPSMAP 9000_Black Box
prweb = prweb.query("~`Project Name`.str.contains('BMW')")
prweb.head()

,Project Name,PM Project Id
0,Force Kayak Propulsion System,10617
1,Rally 110/210 Family,16349
2,Engine Cut-Off System (ECOS/MOB),16198
3,GPSMAP H1/H1i,16137
4,Xero B60i Laser Navigation Binoculars,13229


Merge with data from PMsystem

In [186]:
prweb_pmsystem = pd.merge(left=pmsystem_project_dpqe_melted, right=prweb, how='left', left_on='PM Project Id', right_on='PM Project Id', suffixes=['_PMsystem','_prweb'])

In [187]:
prweb_pmsystem.head()

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score,Project Name
0,7563,Mercury,"Fu, Ernest",Working Prototype Built and Tested,NaN,1,NaN
1,8752,Marine inReach Antenna,"Hsu, Jason(PQE)",Working Prototype Built and Tested,NaN,1,NaN
2,10617,Force Kayak Propulsion System,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2023-10,1,Force Kayak Propulsion System
3,10692,SaMD Sleep Apnea Risk Assessment,"Fu, Ernest",Working Prototype Built and Tested,NaN,1,NaN
4,11252,MARQ Carbon (Gen 2),"Jen, Howard",Working Prototype Built and Tested,NaN,1,MARQ Carbon


In [188]:
Blacklist = 'Mercury|SaMD Sleep Apnea Risk Assessment|GDC 7500|Connect IQ 3.x|GI 275|GLA 85|Tacx NEO Bike Plus|Tacx NEO 3M|Edge 540 840'

In [189]:
prweb_pmsystem = (prweb_pmsystem
 .query("~`PM Project Name`.str.contains(@Blacklist)")
 )


Handle GPSMAP 9000 19"

In [190]:
'Hsu, Jason(PQE);\nWang, Evelyn'.replace('\n','').split(';')

['Hsu, Jason(PQE)', 'Wang, Evelyn']

In [191]:
# GPSMAP 9000 - 19" to Wang, Evelyn, others to Hsu, Jason(PQE)
def gpsmap_9000 (project_name, DPQE):
    if project_name == 'GPSMAP 9000 - 19"':
        return DPQE.replace('\n','').split(';')[1] #Wang, Evelyn
    elif project_name in ['GPSMAP 9000 - 22"','GPSMAP 9000 - 24"','GPSMAP 9000 -27"','GPSMAP 9000_Black Box']:
        return DPQE.replace('\n','').split(';')[0] #Hsu, Jason(PQE)
    else:
        return DPQE


In [192]:
prweb_pmsystem['DPQE'] = np.vectorize(gpsmap_9000)(prweb_pmsystem['Project Name'],prweb_pmsystem['DPQE'])

In [193]:
prweb_pmsystem.drop_duplicates(inplace=True)

In [194]:
#Validate
prweb_pmsystem[prweb_pmsystem['PM Project Name'] == 'GPSMAP 9000']

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score,Project Name
132,9946,GPSMAP 9000,"Wang, Evelyn",PR Date,2023-06,3,"GPSMAP 9000 - 19"""
133,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-06,3,"GPSMAP 9000 - 24"""
134,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-06,3,"GPSMAP 9000 -27"""
135,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-06,3,"GPSMAP 9000 - 22"""
224,9946,GPSMAP 9000,"Wang, Evelyn",MP Date,2023-07,4,"GPSMAP 9000 - 19"""
225,9946,GPSMAP 9000,"Hsu, Jason(PQE)",MP Date,2023-07,4,"GPSMAP 9000 - 24"""
226,9946,GPSMAP 9000,"Hsu, Jason(PQE)",MP Date,2023-07,4,"GPSMAP 9000 -27"""
227,9946,GPSMAP 9000,"Hsu, Jason(PQE)",MP Date,2023-07,4,"GPSMAP 9000 - 22"""


Fill PM system projects' name to PRweb's if PRweb's projects name is Nan (2022/11/11 Ray's request)

In [195]:
def noPRweb(PMProjectName,ProjectName):
    if ProjectName is np.nan:
        return f"1_NoPRweb_{PMProjectName}"
    else:
        return ProjectName

In [196]:
prweb_pmsystem['Project Name'] = np.vectorize(noPRweb)(prweb_pmsystem['PM Project Name'],prweb_pmsystem['Project Name'])

In [197]:
prweb_pmsystem.Date.fillna('1_NoDate(count)', inplace=True)

In [198]:
def weighted_score_is_0_if_NoDate(Date, weighted_score):
    if Date == '1_NoDate(count)':
        return 1
    else:
        return weighted_score
prweb_pmsystem.weighted_score = np.vectorize(weighted_score_is_0_if_NoDate)(prweb_pmsystem.Date, prweb_pmsystem.weighted_score)

Extract MP date and merge to "detail table"

In [199]:
def pmsystem_mp_date(pmsystem_project):
    return(pmsystem_project
    .reset_index()
    .rename(columns={'Project ID':'PM Project Id'})
    .iloc[:,[0,4]]
    .rename(columns = {"MP Date":"MP_Date"})
    .assign(MP_Date = lambda df_ : df_.MP_Date.dt.date)
    )

In [200]:
prweb_pmsystem = pd.merge(prweb_pmsystem, pmsystem_mp_date(pmsystem_project), on='PM Project Id')

In [201]:
detail = (prweb_pmsystem
 .groupby(['DPQE','Project Name','Date','MP_Date'])
 .agg({'weighted_score':'sum'})
#  .agg({'weighted_score':'sum','Project Name':'count'})
 .sort_index(level=['Date','Project Name'])
#  .rename(columns=({'weighted_score':'Weighted_Project_counts','Project Name':'Project_counts'}))
 ['weighted_score']
 .sort_index(level='MP_Date')
 .unstack(level='Date',fill_value=0)
 .style.background_gradient(cmap="OrRd", axis=None)
)
detail

Detail dataframe

In [202]:
# detail = prweb_pmsystem.groupby(['DPQE','Project Name','Date']).agg({'weighted_score':'sum','Project Name':'count'}).sort_index(level=['Date','Project Name']).rename(columns=({'weighted_score':'Weighted_Project_counts','Project Name':'Project_counts'})).unstack(fill_value=0).style.background_gradient(cmap="OrRd", axis=None)
# detail

# Style overview DataFrame

Overview

In [203]:
overview = prweb_pmsystem.groupby(['DPQE','Date']).count()['Project Name'].sort_index(level='Date').unstack(fill_value=0).iloc[:,1:].style.background_gradient(cmap="OrRd", axis=None)
overview

#2022/11/12 iloc[:,1:] is to avoid NoDate projects (count) dominate background_gradient

Date,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02
DPQE,,,,,,,,,,,,,,,,,,,,,
"Chen, Leo(PQE)",0,2,0,0,1,2,0,3,0,1,2,1,3,1,0,0,0,0,0,0,0
"Fu, Ernest",0,0,2,1,2,0,1,2,1,2,0,0,2,1,3,0,0,0,0,0,0
"Hsu, Jason(PQE)",9,7,4,5,5,2,3,3,2,2,2,0,1,0,4,1,1,1,1,1,1
"Jen, Howard",0,1,3,1,0,3,1,6,0,0,6,1,6,0,1,0,1,0,0,0,0
"Kuo, Chris",4,0,3,4,5,3,0,6,0,4,3,0,3,2,1,2,0,0,0,1,0
"Lo, Aalto",1,1,1,0,0,0,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0
"Wang, Cooper",0,1,0,2,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Wang, Evelyn",3,2,4,2,3,1,1,1,2,2,0,3,1,1,0,0,0,0,0,0,0


Overview_weighted

In [204]:
overview_weighted = prweb_pmsystem.groupby(['DPQE','Date']).sum()['weighted_score'].sort_index(level='Date').unstack(fill_value=0).iloc[:,1:].style.background_gradient(cmap="OrRd", axis=None)
overview_weighted

#2022/11/12 iloc[:,1:] is to avoid 1_NoDate(count) dominate background_gradient

C:\Users\wangcooper\AppData\Local\Temp\ipykernel_5292\1236123746.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  overview_weighted = prweb_pmsystem.groupby(['DPQE','Date']).sum()['weighted_score'].sort_index(level='Date').unstack(fill_value=0).iloc[:,1:].style.background_gradient(cmap="OrRd", axis=None)


Date,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02
DPQE,,,,,,,,,,,,,,,,,,,,,
"Chen, Leo(PQE)",0,8,0,0,2,2,0,5,0,2,6,4,11,4,0,0,0,0,0,0,0
"Fu, Ernest",0,0,6,4,8,0,1,2,2,4,0,0,6,3,12,0,0,0,0,0,0
"Hsu, Jason(PQE)",24,26,11,13,14,5,9,6,7,7,8,0,2,0,10,4,4,3,3,4,4
"Jen, Howard",0,3,3,4,0,3,1,12,0,0,18,2,24,0,3,0,4,0,0,0,0
"Kuo, Chris",8,0,11,8,11,5,0,17,0,14,8,0,11,6,4,8,0,0,0,4,0
"Lo, Aalto",3,4,1,0,0,0,2,3,0,8,0,0,0,0,0,0,0,0,0,0,0
"Wang, Cooper",0,2,0,6,8,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Wang, Evelyn",7,8,13,2,12,2,2,2,6,6,0,12,4,4,0,0,0,0,0,0,0


# 2022/09/02 request by Ray

 Find PM system projects MP dates from NOW

In [205]:
pmsystem_project2 = pd.read_excel(pmsystem_project_list_file, usecols=[0,1,5,8,9,10,12], 
    parse_dates=[2,3,4,5], engine='openpyxl')
pmsystem_project2 = pmsystem_project2[pmsystem_project2['Global/Asia']=='Global']
pmsystem_project2.head()

c:\Users\wangcooper\Miniconda3\envs\workspace3.10\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Project ID,Project Name,MP Date,Category,Global/Asia,PDP/Form107,Status
0,16544,2024 Value Dash Cams,2024-03-18,Automotive,Global,PDP,Development
1,15624,Catalyst Kart/Auto,2023-10-23,Automotive,Global,PDP,Development
2,16199,DC-2023 Premium Dash Cameras,2023-08-28,Automotive,Global,PDP,Development
3,16718,DEZL 2024,2024-06-10,Automotive,Global,PDP,Development
4,16249,dezl headset (2nd Gen),2024-05-06,Automotive,Global,PDP,Development


In [206]:
MPdate_from_now = pmsystem_project2[pmsystem_project2['MP Date'] > datetime.now()].sort_values(['Category','MP Date'], ascending=True).set_index('Category')
MPdate_from_now['MP Date'] = MPdate_from_now['MP Date'].dt.date
MPdate_from_now

,Project ID,Project Name,MP Date,Global/Asia,PDP/Form107,Status
Category,,,,,,
Automotive,16199,DC-2023 Premium Dash Cameras,2023-08-28,Global,PDP,Development
Automotive,15624,Catalyst Kart/Auto,2023-10-23,Global,PDP,Development
Automotive,16539,Motorcycle Radar 2023,2024-03-07,Global,PDP,Development
Automotive,16544,2024 Value Dash Cams,2024-03-18,Global,PDP,Development
Automotive,16249,dezl headset (2nd Gen),2024-05-06,Global,PDP,Development
...,...,...,...,...,...,...
Outdoor,16298,Instinct 3 Solar,2024-08-05,Global,PDP,Development
Outdoor,16215,eTrex Touch Gen 2,2024-09-09,Global,PDP,Development
Outdoor,13229,Xero B60i Laser Navigation Binoculars,2024-09-16,Global,PDP,Development


In [207]:
MPdate_from_now_count = MPdate_from_now.groupby('Category')['Project Name'].count().to_frame().rename({'Project Name':'Count'},axis=1)
MPdate_from_now_count

,Count
Category,
Automotive,6
Fitness,13
Garmin Labs,1
Marine,22
OEM Auto,1
Outdoor,26


In [208]:
No_MPdate = pmsystem_project2[pmsystem_project2['MP Date'].isnull()].sort_values(['Category','MP Date'], ascending=True).set_index('Category')
No_MPdate = No_MPdate[~No_MPdate.index.isin(['Aviation','Garmin Labs'])]
No_MPdate

,Project ID,Project Name,MP Date,Global/Asia,PDP/Form107,Status
Category,,,,,,
Automotive,15955,"Tread 8"" OEM - Honda",NaT,Global,PDP,Development
Automotive,16319,Aviation Headset -,NaT,Global,PDP,Concepts
Automotive,17038,Dash Cam Lineup (Gen 7),NaT,Global,PDP,Concepts
Automotive,16808,DEZL Sideview,NaT,Global,PDP,Concepts
Automotive,16717,DriveSmart 67/77/87,NaT,Global,PDP,Concepts
...,...,...,...,...,...,...
Outdoor,16212,Marq (Gen 3),NaT,Global,PDP,Concepts
Outdoor,17061,Micro LED Adventure Watch,NaT,Global,PDP,Concepts
Outdoor,17099,Rino 8xx,NaT,Global,PDP,Concepts


In [209]:
No_MPdate_count = No_MPdate.groupby('Category')['Project Name'].count().to_frame().rename({'Project Name':'Count'},axis=1)
No_MPdate_count

,Count
Category,
Automotive,8
Fitness,12
Marine,13
OEM Auto,21
Outdoor,30


Find Avation projects MP dates from PRweb

In [210]:
prweb2 = pd.read_excel(prweb_file, header=1, usecols=[0,1,5,6])
Aviation_PRweb = prweb2[prweb2['Category'] == 'Aviation'].sort_values('MP Date')
Aviation_PRweb

,Project Name,Project Id,Category,MP Date
79,Product - Flight Stream Redesign,977,Aviation,2023-05-15
39,"Product - 1.3\"" GTR/GNC",984,Aviation,2023-12-18
35,Large Format TXi,1155,Aviation,2024-01-01
36,Product - G3X/G500TXi Hybrid,1050,Aviation,2024-01-01
37,Product - GDU77XW,1048,Aviation,2024-01-01
34,Product - GDU147XW,1049,Aviation,2024-01-03


# to_excel

In [211]:
with pd.ExcelWriter(getcwd()+'\(Output) GMLALoading.xlsx', engine='openpyxl') as writer:

    overview.to_excel(writer, sheet_name='overview')

    overview_weighted.to_excel(writer, sheet_name='overview_weighted')
    
    detail.to_excel(writer, sheet_name='detail')
    #auto_adjust_xlsx_column_width(overall, writer, sheet_name="overall", margin=5)
    
    # MPdate_from_now_count.to_excel(writer, sheet_name='MPdate_from_now_count')
    # auto_adjust_xlsx_column_width(MPdate_from_now_count, writer, sheet_name="MPdate_from_now_count", margin=3)
    
    MPdate_from_now.to_excel(writer, sheet_name='MPdate_from_now')
    auto_adjust_xlsx_column_width(MPdate_from_now, writer, sheet_name="MPdate_from_now", margin=3)
    
    No_MPdate_count.to_excel(writer, sheet_name='No_MPdate_count')
    auto_adjust_xlsx_column_width(No_MPdate_count, writer, sheet_name="No_MPdate_count", margin=3)
    
    No_MPdate.to_excel(writer, sheet_name='No_MPdate')
    auto_adjust_xlsx_column_width(No_MPdate, writer, sheet_name="No_MPdate", margin=3)
    
    pmsystem_project_wo_any_date.to_excel(writer, sheet_name='pmsystem_project_wo_any_date')
    auto_adjust_xlsx_column_width(pmsystem_project_wo_any_date, writer, sheet_name="pmsystem_project_wo_any_date", margin=5)
    
    ww_asia_diff_DPQE.to_excel(writer, sheet_name='ww_asia_diff_DPQE')
    auto_adjust_xlsx_column_width(ww_asia_diff_DPQE, writer, sheet_name="ww_asia_diff_DPQE", margin=5)
    
    projects_wo_DPQE.to_excel(writer, sheet_name='projects_wo_DPQE')
    auto_adjust_xlsx_column_width(projects_wo_DPQE, writer, sheet_name="projects_wo_DPQE", margin=5)
    
    multi_dpqe.to_excel(writer, sheet_name='multi_dpqe')
    auto_adjust_xlsx_column_width(multi_dpqe, writer, sheet_name="multi_dpqe", margin=5)


    Aviation_PRweb.to_excel(writer, sheet_name='Aviation_PRweb')
    auto_adjust_xlsx_column_width(Aviation_PRweb, writer, sheet_name="Aviation_PRweb", margin=3)

